In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License"); you may
# not use this file except in compliance with the License. You may obtain
# a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS, WITHOUT
# WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the
# License for the specific language governing permissions and limitations
# under the License.

# Watson Studioで文字データを可視化しよう

twitterから気になる情報を取得し、 Watson Natural Languege UnderstandingでKeyword抽出、WorldCloudをPixieDust を使用して表示してみましょう!

<a id="part1"></a>
# Part 1 - 分析データ作成
<a id="setup"></a>
## 1. Setup
### 1.1 最新の Watson Developer Cloud, requests-oauthlib パッケージの導入
Natural　Languge　Understandingに使用するWatson Developer CloudとTwitterのOAuth認証に使用する requests-oauthlibパッケージを導入します。

In [ ]:
!pip install --upgrade ibm-watson

!pip install requests requests_oauthlib

<a id="pixie"></a>
### 1.2 PixieDust Libraryの導入
このノートブックでは、PixieDustライブラリを使用してデータセットを分析および視覚化します。

PixieDustの詳細は[Introductory Notebook](https://dataplatform.cloud.ibm.com/exchange/public/entry/view/5b000ed5abda694232eb5be84c3dd7c1) または [PixieDust Github](https://ibm-cds-labs.github.io/pixiedust/)　を参照してください。


次のセルを実行して、最新バージョンのPixieDustを実行していることを確認します。 ローカルのjupyter notyebookを使用し、PixieDustをローカルにインストール済みで、それを使用したい場合は、このセルを実行しないでください。

尚、正式リリース前の`https://github.com/pixiedust/pixiedust.git@va-working-branch#egg=pixiedust`はフォント指定が可能なモジュールで、正式リリースまで一時的に利用しています。日本語表示を可能にするために使用しています。

In [ ]:
# To confirm you have the latest version of PixieDust on your system, run this cell
#!pip install -U --no-deps pixiedust
!pip install --upgrade --no-deps git+https://github.com/pixiedust/pixiedust.git@va-working-branch#egg=pixiedust

PixieDustをインポートし、カーネルをRestartが必要な場合はRestartさせます。

In [ ]:
import pixiedust

 #### 1.4　Option
 <span style="color: red">Pixiedust runtime updated. Please restart kernel</span>  と表示された場合は、上のメニューの`Kernel`-> `Restart`からカーネルをRestartさせてください。

<a id="wordcloud"></a>
### 1.3 wordcloud Libraryの導入

日本語が表示できるように、日本語フォントも導入します。

In [ ]:
!pip install --user wordcloud

In [ ]:
#日本語フォントの導入
jp_font_path ='/home/dsxuser/work/ipaexg00301/ipaexg.ttf'

import os
if not os.path.exists(jp_font_path):
    !wget https://oscdl.ipa.go.jp/IPAexfont/ipaexg00301.zip
    !unzip ipaexg00301.zip
else:
    print('IPA font haｓ been already installed')
 

<a id="pixie_wordcloud"></a>
### 1.4 PixieDust にWordCloudの設定
PixieDustにWordCloud形式でデータが表示できる チャートを追加しましょう。
PixieDustは自分で設定したフォーマットでデータ表示するチャートを設定することができます。

今回は１列目にwordcloudに表示する文字、２列目にその表示Volume数を入れたpandasのDataFrameを渡すと、wordcloudを表示するチャートを設定します。
例えば下記のようなデータです:

```
import pandas as pd
df = pd.DataFrame([["四月", 26],["May", 10],["June", 5]],  columns=['key', 'value'])
```
| 　 |  Key  | Value |
| ---- | ---- | ---- |
|  0  |  四月  | 26 |
|  1  |  May  | 10 |
|  2  |  June  | 5 |

In [ ]:
from pixiedust.display.display import *
import io
import base64
from wordcloud import WordCloud

class SimpleWordCloudDisplay(Display):
    def doRender(self, handlerId):
        # convert from dataframe to dict
        dfdict = {}
       # df = self.entity.toPandas()
        df = self.entity
        for x in range(len(df)):
            currentid = df.iloc[x,0] or 'NoKey'
            currentvalue = df.iloc[x,1]
            dfdict.setdefault(currentid, 0)
            dfdict[currentid] = dfdict[currentid] + currentvalue

        # create word cloud from dict
        wc = WordCloud(background_color="white",  width=800, height=400, max_font_size=140, font_path=jp_font_path).fit_words(dfdict)
        #wc = WordCloud(background_color="white", max_font_size=140, font_path=jp_font_path).fit_words(dfdict)


        # encode word cloud image to base64 string
        img = wc.to_image()
        buffer =io.BytesIO()
        img.save(buffer,format="JPEG")                  #Enregistre l'image dans le buffer
        myimage = buffer.getvalue()  
        img_str = base64.b64encode(myimage)
      

        self._addHTMLTemplateString(
"""
<center><img src="data:image/png;base64,{0}"></center>
""".format(img_str.decode("ascii"))
            
        )

In [ ]:
@PixiedustDisplay()
class SimpleWordCloudMeta(DisplayHandlerMeta):
    @addId
    def getMenuInfo(self,entity,dataHandler):
        if entity.__class__.__name__ == "DataFrame":
            return [
                {
                    "categoryId": "Chart",
                    "title": "Simple Word Cloud",
                    "icon": "fa-cloud",
                    "id": "mySimpleWordCloud"
                }
            ]
        else:
            return []

    def newDisplayHandler(self,options,entity):
        return SimpleWordCloudDisplay(options,entity)

### 1.5 WordCloudのTest
表示できるか確認してみましょう！

In [ ]:
#Test Code
import pandas as pd
jp_font_path='/home/dsxuser/work/ipaexg00301/ipaexg.ttf'
df = pd.DataFrame([["四月", 26],["May", 10],["June", 5]],  columns=['key', 'value'])
display(df, font_path=jp_font_path)

<a id="setupenv"></a>
## 2. 環境準備

以下の3つができるように環境の設定を行います。

1:  Twitter APIを使用してTweet Dataを取得します。
Twitter API KEYの取得が必要です。ここではその取得方法は説明しませんので、お持ちでない方は「Twitter API KEY 取得」等で検索し、取得お願いします。以下の4つの値が必要です。

* AccessToken
* AccessTokenSecret
* ConsumerKey
* ConsumerSecret

2:  IBM Cloud Natural Language Understanding サービスでTweetを分析します。
Natural Language Understanding サービスの作成しAPIKEY,URLの値が必要です。
Natural Language Understanding サービスが未作成の方は、[こちら](https://cloud.ibm.com/catalog/services/natural-language-understanding)より作成してください。

3: 設定ファイルや作成したファイルをIBM Cloud Object Strageを使用してて読み込み、保存します。
IBM Cloud Object Strageを使用する設定を以下で行います。

### 2.1 apikeys.iniの作成
準備した上記の値をapikeys.iniというファイルを自分のPC上で作成し、以下のように値を記入して保存してください。
[　]でかこまれた部分を自分のKEYに変更します。[　]は不要ですので残さないようにしてください。
```
[TWITTER]
TWITTER_AccessToken=[AccessTokenの値]
TWITTER_AccessTokenSecret=[AccessTokenSecretの値]
TWITTER_ConsumerKey=[ConsumerKeyの値]
TWITTER_ConsumerSecret=[ConsumerSecreの値]
[IBM]
NLU_APIKEY=[Natural Language UnderstandingサービスのAPIの値]
NLU_URL=[Natural Language UnderstandingサービスのURLの値]
```

### 2.2 apikeys.iniのアップロード
Watson Studioのjupyter note book上、右上の`0101`というアイコンをクリックし、Fileアップロードの画面を出します。`Drop your file here or browse your files to add a new file` と書いてある場所に、2.1で作成したファイルをドロップし、Watson Studio上のProjectにアップロードします。

<a id='load'></a> 
### 2.3 apikeys.iniの読み込み

　1. **下のセルを選択して、空の行にカーソルを置いてください。** 

2. アップロードしたTweeterAPIKey.iniファイルの下にある(見えない場合は右上の10/01アイコンをクリック) `Insert to code`の下にある`Insert StreamingBody object`をクリックしてください。

3. ファイルを読み込むストリーム`streaming_body_2`をセットするコードが挿入されます。

4.  4箇所ある`streaming_body_2`は　全て`streaming_body_1`に変更します。(後のコードで使用するため)

<p><span style="color: teal">
# Your data file was loaded into a botocore.response.StreamingBody object.<br/>
# Please read the documentation of ibm_boto3 and pandas to learn more about your possibilities to load the data.<br/>
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/<br/>
# pandas documentation: http://pandas.pydata.org/<br/></span>
<strong>streaming_body_1</strong> = client_xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx.get_object(Bucket='wordcloud-donotdelete-pr-wyztdevipqhfkt', Key='apikeys.ini')['Body']</p>
<p><span style="color: teal">
 # add missing __iter__ method, so pandas accepts body as file-like object</span>
if not hasattr(<strong>streaming_body_2</strong>, "__iter__"): <strong>streaming_body_2</strong>.__iter__ = types.MethodType( __iter__, <strong>streaming_body_2</strong> ) 
</p>


## <span style="color: red">この下に入力 </span>

In [ ]:
# この行の下にカーソルを置いて、Insert StreamingBody objectをクリック



### 2.4 ConfigParserへの設定情報の読み込み

In [ ]:
import configparser

inifile = configparser.ConfigParser(interpolation=configparser.ExtendedInterpolation())
inifile.read_string(streaming_body_1.read().decode('utf-8'))

### 2.5 IBM Objerct StrageのCredentialセット
結果のファイル保存に使用するため、IBM Objerct StrageのCredentialをセットします。

　1. **下のセルを選択して、空の行にカーソルを置いてください。** 

2. アップロードしたTweeterAPIKey.iniファイルの下にある(見えない場合は右上の10/01アイコンをクリック) `Insert to code`の下にある`Insert Credentials`をクリックしてください。

3. ファイルを読み込むストリーム`credentials_2`をセットするコードが挿入されます。

4.  4箇所ある`streaming_body_2`は　全てcredentials_1`に変更します。(後のコードで使用するため)

<p><span style="color: teal">
# @hidden_cell<br/>
# The following code contains the credentials for a file in your IBM Cloud Object Storage.<br/>
# You might want to remove those credentials before you share your notebook.<br/>
</span>
<strong> credentials_2</strong>  = {
    'IAM_SERVICE_ID': 'iam-ServiceId-xxxxxxxx-xxxx-xxxx-xxxx-1234567890xx',<br/>
    'IBM_API_KEY_ID': 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',<br/>
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',<br/>
    'IBM_AUTH_ENDPOINT': 'https://iam.bluemix.net/oidc/token',<br/>
    'BUCKET': 'wordcloud-donotdelete-pr-wyztdevipqhfkt',<br/>
    'FILE': 'apikeys.ini'<br/>
}</p>

## <span style="color: red">この下に入力 </span>

In [ ]:
# この行の下にカーソルを置いて、Insert　Credentialsをクリック


<a id="gettweets"></a>
## 3. Tweet情報の取得と保存

### 2.1 パラメータの設定
検索したい文字列、取得したいTweet数、検索の繰り返し回数をセット。
1回の検索で最大100 件しか取得できないので、それ以上取得したい場合は、繰り返し回数をセットする。

In [ ]:
#以下のパラメータをセットする
#検索したい文字列
#基本は最新1週間分
search_key = "#BMXUG"

#1回の検索で取得したいTweet数, 最大は１００
search_count=100

#検索の繰り返し回数
repeat_count=2

#search_until="2019-04-07" #期間を指定したい場合はコメントをはずす

### 2.2 Tweet情報の取得
Twitter Search APIを使って、Tweet情報を取得します。

In [ ]:
import json
import urllib
import calendar
import time
from requests_oauthlib import OAuth1Session

CK = inifile['TWITTER']['TWITTER_ConsumerKey']
CS =  inifile['TWITTER']['TWITTER_ConsumerSecret']
AT =  inifile['TWITTER']['TWITTER_AccessToken']
ATS =  inifile['TWITTER']['TWITTER_AccessTokenSecret']
twitter = OAuth1Session(CK, CS, AT, ATS)  
search_endpoint = "https://api.twitter.com/1.1/search/tweets.json"



def search_tw(tw, params, search_endpoint):

    url = "https://api.twitter.com/1.1/search/tweets.json"  # 取得エンドポイント
    res = tw.get(search_endpoint, params = params)

    if res.status_code == 200:  # 正常通信出来た場合
        timelines = json.loads(res.text)  # レスポンスからタイムラインリストを取得
        df_tweets = pd.DataFrame.from_dict(timelines.get('statuses'), dtype='object')
        next_param = timelines.get('search_metadata').get('next_results')
        if next_param is not None:
            next_param = urllib.parse.parse_qs(next_param.replace('?', ''))
        return df_tweets, next_param

    else:  # 正常通信出来なかった場合
        print("Failed: %d" % res.status_code)
        raise ValueError("Failed: %d" % res.status_code)
        
def YmdHMS(created_at):
    time_utc = time.strptime(created_at, '%a %b %d %H:%M:%S +0000 %Y')
    unix_time = calendar.timegm(time_utc)
    time_local = time.localtime(unix_time)
    return str(time.strftime("%Y-%m-%d %H:%M:%S", time_local))
               
param_dict = {} 
param_dict['q'] = search_key
#param_dict['until'] = search_until    #期間を指定したい場合はコメントをはずす
param_dict['result_type'] = 'recent'
param_dict['count'] = search_count
param_dict['lang'] = 'ja'

next_params = param_dict
df_tweets= pd.DataFrame()
last_id = ''
for i in range(int(repeat_count)):
    print('repaet#:{}'.format(i))
    df, next_params = search_tw(twitter, next_params, search_endpoint)
    df_tweets = pd.concat([df_tweets, df], ignore_index=True,)
    if len(df) == 0:
        break
    if next_params is None:
        before_last_id = last_id
        last_id = df.tail(1)['id'].astype(str).values[0]
        if before_last_id == last_id:
            break
        param_dict['max_id'] = last_id
        next_params = param_dict
if len(df_tweets) > 0:
    df_tweets['create_at_jst'] = df_tweets.apply(lambda x: YmdHMS(x['created_at']) if x.dtype == "object" else x, axis=1)
    df_tweets = df_tweets.sort_values(by=["create_at_jst"], ascending=True)
    col_list = list(df_tweets.columns.values)
    del col_list[-1]
    col_list.insert(0, 'create_at_jst')
    df_tweets = df_tweets.reindex(columns=col_list)
else:
    print('No Data')


### 2.3 Tweet情報の取得
結果を一旦ファイルに保存します。

In [ ]:
#IBM Object Storageへの検索保存
tweet_result_filename = 'tweets_resluts.csv'

cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_1['ENDPOINT'])


# Write a CSV file from the enriched pandas DataFrame.
df_tweets.to_csv(tweet_result_filename, index=False)

# Use the above put_file method with credentials to put the file in Object Storage.
cos.upload_file(tweet_result_filename, Bucket=credentials_1['BUCKET'],Key=tweet_result_filename)

<a id="donulu"></a>
## 4. TweetからNatural Language Understandingを使用してKeyword抽出

### 4.1 前処理後NLU呼び出しと合計処理

In [ ]:
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, KeywordsOptions


df_tweets['refineText']=df_tweets['text']

# 改行削除
df_tweets['refineText'] = df_tweets.apply(lambda x: x['refineText'].replace('\n', ' '), axis=1)
df_tweets['refineText'] = df_tweets.apply(lambda x: x['refineText'].strip(), axis=1)

# RTマーク削除
df_tweets['refineText'] = df_tweets['refineText'].replace(r'^RT.*?:', '', regex=True)

# hasshutag記号(#)削除
df_tweets['refineText'] = df_tweets['refineText'].replace(r'#(\w+)', '', regex=True)

# Strip links from Text column
df_tweets['refineText'] = df_tweets['refineText'].apply(lambda x: x.split('http')[0])

# カッコ削除
df_tweets['refineText'] = df_tweets.apply(lambda x: x['refineText'].replace('(', ' '), axis=1)
df_tweets['refineText'] = df_tweets.apply(lambda x: x['refineText'].replace(')', ' '), axis=1)
df_tweets['refineText'] = df_tweets.apply(lambda x: x['refineText'].replace('【', ' '), axis=1)
df_tweets['refineText'] = df_tweets.apply(lambda x: x['refineText'].replace('】', ' '), axis=1)
df_tweets['refineText'] = df_tweets.apply(lambda x: x['refineText'].replace('[', ' '), axis=1)
df_tweets['refineText'] = df_tweets.apply(lambda x: x['refineText'].replace(']', ' '), axis=1)
df_tweets['refineText'] = df_tweets.apply(lambda x: x['refineText'].replace('「', ' '), axis=1)
df_tweets['refineText'] = df_tweets.apply(lambda x: x['refineText'].replace('」', ' '), axis=1)


nlu = NaturalLanguageUnderstandingV1(version='2018-11-16', url=inifile['IBM']['NLU_URL'], iam_apikey=inifile['IBM']['NLU_APIKEY'])

features = Features(keywords=KeywordsOptions())
keywords = []
df_pixiedust = pd.DataFrame(columns=['Key', 'Value'])


for text, i in zip(df_tweets['refineText'], df_tweets.index):
    if not text:
        keywords.append(' ')
        continue

    enriched_json = nlu.analyze(text=text, features=features, language='ja')

    # Iterate and get KEYWORDS with a confidence of over 70%
    if 'keywords' in enriched_json.result:
        tmpkw = []
        for kw in enriched_json.result["keywords"]:
            if (float(kw["relevance"]) >= 0.7):
                tmpkw.append(kw["text"])
                df_pixiedust = df_pixiedust.append({'Key': kw["text"], 'Value': 1}, ignore_index=True)
        # Convert multiple keywords in a list to a string and append the string
        keywords.append(', '.join(tmpkw))
    else:
        keywords.append("")
    print('{}:keyword: {}'.format(i, tmpkw))

df_tweets['Keywords'] = keywords

df_pixiedust = df_pixiedust.groupby('Key').count()
df_pixiedust = df_pixiedust.sort_values(by=["Value"], ascending=False)
df_pixiedust = df_pixiedust.reset_index()

# Write a CSV file from the enriched pandas DataFrame.
df_tweets.to_csv(tweet_result_filename, index=False)

# Use the above put_file method with credentials to put the file in Object Storage.
cos.upload_file(tweet_result_filename, Bucket=credentials_1['BUCKET'],Key=tweet_result_filename)

### 4.2　抽出データ確認

In [ ]:
df_pixiedust

### 4.3 WordCloudで表示してみましょう!

In [ ]:
display(df_pixiedust, font_path = '/home/dsxuser/work/ipaexg00301/ipaexg.ttf')
